#Churn Prediction

##Módulo 5: Caso Práctico individual
###Alumno: 131044

![churn](http://chaotic-flow.com/wp-content/uploads/2017/09/saas-churn-leaky-bucket.png)

Este proyecto corresponde al módulo de Data Science del Master en Big Data Management. Por lo tanto, **el objetivo es el siguiente:** Poder predecir quien va a causar baja como cliente de una Telco en función de sus servicios contratados, tiempo en la compañía, facturación y tipos de contratos. 

Los datos a utilizar proceden de Kaggle (https://www.kaggle.com/blastchar/telco-customer-churn) y contienen información sobre clientes, productos y servicios contratados, información contractual, información método de pago y facturación, información permanencia clientes e información abandono de clientes.

La predicción se llevará a cabo estudiando previamente el dataset mediante un análisis exploratorio y la posterior elección del algoritmo ás adecuado para la solución del problema.

In [0]:
import pandas as pd 
import numpy

## ACCESO A LOS DATOS

In [12]:
dataset = pd.read_csv("churn.csv", sep=",",decimal=".")
dataset.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


##LIMPIEZA DE LOS DATOS

In [13]:
binario = {'Yes': 1,'No': 0,'No internet service':0,'No phone service':0,'Female':1,'Male':0,'':'null',' ':'null'} 
dataset.gender = [binario[item] for item in dataset.gender] 
dataset.Partner = [binario[item] for item in dataset.Partner] 
dataset.Dependents =  [binario[item] for item in dataset.Dependents] 
dataset.PhoneService =  [binario[item] for item in dataset.PhoneService] 
dataset.MultipleLines =  [binario[item] for item in dataset.MultipleLines] 
dataset.OnlineBackup =  [binario[item] for item in dataset.OnlineBackup] 
dataset.DeviceProtection =  [binario[item] for item in dataset.DeviceProtection] 
dataset.StreamingTV =  [binario[item] for item in dataset.StreamingTV] 
dataset.StreamingMovies =  [binario[item] for item in dataset.StreamingMovies] 
dataset.PaperlessBilling =  [binario[item] for item in dataset.PaperlessBilling] 
dataset.Churn =  [binario[item] for item in dataset.Churn] 
dataset.TechSupport	 =  [binario[item] for item in dataset.TechSupport]
dataset.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,1,0,1,0,0,DSL,No,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,0,0,0,0,34,1,0,DSL,Yes,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,0,0,0,0,2,1,0,DSL,Yes,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,0,0,0,0,45,0,0,DSL,Yes,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,1,0,0,0,2,1,0,Fiber optic,No,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null int64
SeniorCitizen       7043 non-null int64
Partner             7043 non-null int64
Dependents          7043 non-null int64
tenure              7043 non-null int64
PhoneService        7043 non-null int64
MultipleLines       7043 non-null int64
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null int64
DeviceProtection    7043 non-null int64
TechSupport         7043 non-null int64
StreamingTV         7043 non-null int64
StreamingMovies     7043 non-null int64
Contract            7043 non-null object
PaperlessBilling    7043 non-null int64
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null int64
dtypes: float64(1), int64(14), object(6)
memo

###Comprobación de nulls y blancos en R

Ya que la función is.na() de R, detecta tanto los valores nulos como las celdas vacías, se ha decidido hacer una doble comprobación en R, ya que este tipo de valores caussarán errores en los siguientes pasos del proyecto.

**Input:** dataset %>% summarise_all(funs(sum(is.na(.))))

**output:**
TotalCharges = 11
Resto de Variables = 0

R detecta 11 NAs en la columna "TotalCharges" mientras que en Python no existe ningún valor nulo en esta viariable. Por  lo tanto,  pordríamos asumir que esos NAs corresponden a celdas vacías. En este caso, esos 11 clientes son nuevos y no tienen ninguna cuota pagada,ya que ternure para esos valores es 0 por lo tanto tendría sentido que estén vacíos aun, pero para que no nos de errores en los siguientes pasos se le asigna el valor 0.

In [0]:
# Intentar convertir los vacios en 0
# dataset.TotalCharges.replace('',0)
# dataset.TotalCharges.replace(' ',0)
# numpy.where(dataset["TotalCharges"]=='29.85')
# dataset["TotalCharges"].unique()
#dataset.TotalCharges = pd.to_numeric(dataset.TotalCharges, errors='coerce')
#dataset.info()
#numpy.where(dataset["TotalCharges"]=='29.85')
numpy.where(dataset["customerID"]=='')


NameError: ignored

In [0]:
dataset.describe().round()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,Churn
count,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0
mean,0.0,0.0,0.0,0.0,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,65.0,0.0
std,1.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0
25%,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0
50%,0.0,0.0,0.0,0.0,29.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,70.0,0.0
75%,1.0,0.0,1.0,1.0,55.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,90.0,1.0
max,1.0,1.0,1.0,1.0,72.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,119.0,1.0


In [0]:
# Generamos el histograma de Age por Sex:
dataset.loc[:,['Churn','Sex']].hist(bins=15,by='Sex',sharey=True)

# Representamos la gráfica:
plt.show()

# MODELIZACIÓN

Se va a llevar a cabo el siguiente proceso:

 1. Probar con 3 modelos distintos (Gaussian, Decision Tree y AdaBoost)
 2. Evaluar las metricas de los 3 modelos en función de las métricas que definamos
 3. Para el modelo que más nos interese, lanzar un proceso de tunning
 4. De todos los modelos lanzados, evaluar cual es el ideal en función de las curvas de performance
 
Lo siguiente que vamos a hacer es eliminar la columa 'Survived' del dataset y marcarla como outcome

In [44]:
pip install -U scikit-learn

     |████████████████████████████████| 6.7MB 2.2MB/s 
  Found existing installation: scikit-learn 0.20.dev0
    Uninstalling scikit-learn-0.20.dev0:
      Successfully uninstalled scikit-learn-0.20.dev0


In [1]:
# 'churn' será nuestra etiqueta y el valor que queremos predecir:
from sklearn.cross_validation import train_test_split
outcomes = dataset['Churn']
data = dataset.drop(['Churn'], axis = 1)

# Representar las primeras 5 filas para comprobar los cambios
data.head()

ModuleNotFoundError: ignored

A continuación, generar los conjuntos de train y test:

In [42]:
# TODO: Particionar data y outcome para crear los sets de train y test. Fijar un random_state y un tamaño del test del 0.2 
X_train, X_test, y_train, y_test = train_test_split(data, outcomes)

# Mostrar resultados
print("Training set tiene {} samples.".format(X_train.shape[0]))
print("Testing set tiene {} samples.".format(X_test.shape[0]))

NameError: ignored

Ahora vamos a definir una función para realizar el entrenamiento:

In [0]:
from sklearn.metrics import fbeta_score, accuracy_score
from time import time

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: entrena el modelo seleccionando los datos en función de 'sample_size'
    start = time() # Obtener el tiempo de comienzo
    learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = time() # Obtener el tiempo de final de entrenamiento
    
    # TODO: estimar tiempo de entrenamiento
    results['train_time'] = end-start
        

        
    # TODO: Obtener predicciones en test set,
    #       despues, generar predicciones para los 300 primeros samples
    start = time() # comienzo
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # fin
    
    # TODO: prediction time
    results['pred_time'] = end-start
            
    # TODO: accuracy en los primeros 300 samples
    results['acc_train'] = accuracy_score(y_train[:300],predictions_train)
    
    # TODO: accuracy on test set
    results['acc_test'] = accuracy_score(y_test,predictions_test)
            
    # TODO: Compute F-score en los primeros 300 training samples
    results['f_train'] = fbeta_score(y_train[:300],predictions_train,beta=0.5)
        
    # TODO: Compute F-score en el test set
    results['f_test'] = fbeta_score(y_test,predictions_test,beta=0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

Y vamos a implementar la función:

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import visuals as vs

# TODO: Initializa los 3 modelos fijando el random_state cuando se pueda
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_1 = int(len(X_train)/70)
samples_10 = int(len(X_train)/20)
samples_100 = int(len(X_train))

# Obtener resultados
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

print(X_train.shape)
vs.evaluate(results)

In [0]:
Para cada uno de los modelos lanzados, obten su curva de aprendizaje asociada. ¿Eres capaz de decir cual esta en Overtiffing, underfitting u optimo?

In [0]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

title = "Learning Curve"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = clf_C
vs.plot_learning_curve(estimator, title, X_train, y_train, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

Vamos a ajustar los parametros de nuestro modelo seleccionado para mejorar su performance

In [0]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# TODO: Initialize the classifier
clf = AdaBoostClassifier(random_state=0)

# TODO: Create the parameters list you wish to tune
#parameters = {'n_estimators':[75,100,200]}
parameters = {'n_estimators':[75,200],'learning_rate':[1.0,1.5,2.0]}

# TODO: Make an fbeta_score scoring object
scorer = make_scorer(fbeta_score, beta=0.5)
# TODO: Perform grid search on the classifier using 'scorer' as the scoring method
grid_obj = GridSearchCV(clf, parameters,scoring=scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

Algorithmos que son del tipo "Árboles" como AdaBoost o Decission Tree nos pueden mostrar cuales son las variables más importantes utilizadas por el modelo:

In [0]:
# TODO: Train the supervised model on the training set 
#AdaBoostClassifier(random_state=0)
model = clf_B.fit(X_train, y_train)

# TODO: Extract the feature importances
importances = model.feature_importances_

# Plot
vs.feature_plot(importances, X_train, y_train)

Vamos a utilizar las 2 variables más importantes para visualizar como funciona el arbol:

In [0]:
%load_ext autoreload
%autoreload 2

data_criterio = dataset.loc[(dataset['Sex']=='female')&(dataset['Fare']>100), :]
vs.aplicar_criterio(dataset, data_criterio, 'Survived')

In [0]:
Ahora vamos a representar el arbol de decision:

In [0]:
from sklearn.externals.six import StringIO
import pydotplus 
from sklearn import tree
print(data.columns)


dot_data = tree.export_graphviz(clf_B, out_file=None,
                     feature_names=data.columns,
                     class_names=['No sobrevive','Sobrevive'],
                     filled=True, rounded=True,
                     proportion=True,
                     special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png(), width=4000, height=4000)